In [1]:
from MotionGPT import *

In [2]:
loader = FBXDataLoader("FBXData")

In [3]:
loader.LoadData()

FBXDataLoader loading data...


In [4]:
len(loader.raw_data)

232

In [5]:
len(loader.raw_data[0])

24

In [6]:
loader.PrepareTrainingData(12)

In [7]:
len(loader.train_data)

2052

In [8]:
len(loader.train_data[180])

4

In [9]:
for bd,pd in loader.next_batch():
    print(bd.shape, pd.shape)


torch.Size([16, 35, 66]) torch.Size([16, 35])
torch.Size([16, 22, 66]) torch.Size([16, 22])
torch.Size([16, 29, 66]) torch.Size([16, 29])
torch.Size([16, 28, 66]) torch.Size([16, 28])
torch.Size([16, 27, 66]) torch.Size([16, 27])
torch.Size([16, 28, 66]) torch.Size([16, 28])
torch.Size([16, 21, 66]) torch.Size([16, 21])
torch.Size([16, 24, 66]) torch.Size([16, 24])
torch.Size([16, 35, 66]) torch.Size([16, 35])
torch.Size([16, 49, 66]) torch.Size([16, 49])
torch.Size([16, 27, 66]) torch.Size([16, 27])
torch.Size([16, 23, 66]) torch.Size([16, 23])
torch.Size([16, 31, 66]) torch.Size([16, 31])
torch.Size([16, 40, 66]) torch.Size([16, 40])
torch.Size([16, 32, 66]) torch.Size([16, 32])
torch.Size([16, 25, 66]) torch.Size([16, 25])
torch.Size([16, 34, 66]) torch.Size([16, 34])
torch.Size([16, 22, 66]) torch.Size([16, 22])
torch.Size([16, 51, 66]) torch.Size([16, 51])
torch.Size([16, 14, 66]) torch.Size([16, 14])
torch.Size([16, 50, 66]) torch.Size([16, 50])
torch.Size([16, 30, 66]) torch.Siz

In [10]:
batch_data, pad_data = next(loader.next_batch())

In [11]:
batch_data.shape

torch.Size([16, 35, 66])

In [12]:
pad_data.shape

torch.Size([16, 35])

In [13]:
from VRNN import *

In [14]:
input_dim = len(loader.train_data[0][1])
h_dim = 64
z_dim = 10
n_layers = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  #device

In [15]:
print(device)

cuda


In [16]:
model = VRNN(input_dim, h_dim, z_dim, n_layers, device)
model.to(device)

VRNN(
  (phi_x): Sequential(
    (0): Linear(in_features=66, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
  )
  (phi_z): Sequential(
    (0): Linear(in_features=10, out_features=64, bias=True)
    (1): ReLU()
  )
  (enc): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
  )
  (enc_mean): Linear(in_features=64, out_features=10, bias=True)
  (enc_std): Sequential(
    (0): Linear(in_features=64, out_features=10, bias=True)
    (1): Softplus(beta=1, threshold=20)
  )
  (prior): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
  )
  (prior_mean): Linear(in_features=64, out_features=10, bias=True)
  (prior_std): Sequential(
    (0): Linear(in_features=64, out_features=10, bias=True)
    (1): Softplus(beta=1, threshold=20)
  )
  (dec): Sequential(
    (0): Linear(in

In [17]:
batch_size = 16
n_epochs = 100
clip = 10
learning_rate = 1e-3
print_every = 10

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
for epoch in range(n_epochs):
    batch_idx = 0   
    for batch_data, pad_data in loader.next_batch(batch_size):
        batch_idx += 1

        batch_data = batch_data.to(device)
        pad_data = pad_data.to(device)

        optimizer.zero_grad()

        kld_loss, nll_loss, _, _ = model(batch_data, pad_data)
        loss = kld_loss + nll_loss
        loss.backward()

        #grad norm clipping, only in pytorch version >= 1.10
        torch.nn.utils.clip_grad_norm(model.parameters(), clip)
        optimizer.step()

        #printing
        if batch_idx % print_every == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t KLD Loss: {:.6f} \t NLL Loss: {:.6f}'.format(
                epoch, batch_idx * len(batch_data), len(loader.train_data),
                100. * batch_idx / len(loader.train_data),
                kld_loss.data / batch_size,
                nll_loss.data / batch_size))

D:\Anaconda3\envs\Transformer\lib\site-packages\ipykernel_launcher.py:16: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  app.launch_new_instance()


Train Epoch: 0 [160/2052 (0%)]	 KLD Loss: 0.775364 	 NLL Loss: 11284.482422
Train Epoch: 0 [320/2052 (1%)]	 KLD Loss: 61.307632 	 NLL Loss: 46953.015625
Train Epoch: 0 [480/2052 (1%)]	 KLD Loss: 640.820374 	 NLL Loss: 24631.425781
Train Epoch: 0 [640/2052 (2%)]	 KLD Loss: 846.595398 	 NLL Loss: 4321.457520
Train Epoch: 0 [800/2052 (2%)]	 KLD Loss: 544.751892 	 NLL Loss: 6188.418945
Train Epoch: 0 [960/2052 (3%)]	 KLD Loss: 286.035034 	 NLL Loss: 7950.365723
Train Epoch: 0 [1120/2052 (3%)]	 KLD Loss: 909.909241 	 NLL Loss: 71602.023438
Train Epoch: 0 [1280/2052 (4%)]	 KLD Loss: 754.500366 	 NLL Loss: 6348.001953
Train Epoch: 0 [1440/2052 (4%)]	 KLD Loss: 254.184784 	 NLL Loss: 22723.671875
Train Epoch: 0 [1600/2052 (5%)]	 KLD Loss: 596.421448 	 NLL Loss: 394089.812500
Train Epoch: 0 [1760/2052 (5%)]	 KLD Loss: 33.061310 	 NLL Loss: 1024.713379
Train Epoch: 0 [1920/2052 (6%)]	 KLD Loss: 315.718536 	 NLL Loss: 21988.535156
Train Epoch: 1 [160/2052 (0%)]	 KLD Loss: 25.335154 	 NLL Loss: 96

Train Epoch: 9 [160/2052 (0%)]	 KLD Loss: 6.784226 	 NLL Loss: 8458.011719
Train Epoch: 9 [320/2052 (1%)]	 KLD Loss: 48.419247 	 NLL Loss: 13470.113281
Train Epoch: 9 [480/2052 (1%)]	 KLD Loss: 29.720669 	 NLL Loss: 9024.613281
Train Epoch: 9 [640/2052 (2%)]	 KLD Loss: 5.617400 	 NLL Loss: 2699.017822
Train Epoch: 9 [800/2052 (2%)]	 KLD Loss: 7.611531 	 NLL Loss: 4131.024902
Train Epoch: 9 [960/2052 (3%)]	 KLD Loss: 16.264359 	 NLL Loss: 5613.184570
Train Epoch: 9 [1120/2052 (3%)]	 KLD Loss: 16.467695 	 NLL Loss: 69748.625000
Train Epoch: 9 [1280/2052 (4%)]	 KLD Loss: 28.231785 	 NLL Loss: 4300.016113
Train Epoch: 9 [1440/2052 (4%)]	 KLD Loss: 8.500415 	 NLL Loss: 22605.949219
Train Epoch: 9 [1600/2052 (5%)]	 KLD Loss: 78.848373 	 NLL Loss: 385466.687500
Train Epoch: 9 [1760/2052 (5%)]	 KLD Loss: 2.881235 	 NLL Loss: 892.759766
Train Epoch: 9 [1920/2052 (6%)]	 KLD Loss: 40.571602 	 NLL Loss: 18798.976562
Train Epoch: 10 [160/2052 (0%)]	 KLD Loss: 14.376349 	 NLL Loss: 6894.723145
Train

Train Epoch: 17 [1760/2052 (5%)]	 KLD Loss: 3.297612 	 NLL Loss: 937.329468
Train Epoch: 17 [1920/2052 (6%)]	 KLD Loss: 39.766350 	 NLL Loss: 13618.395508
Train Epoch: 18 [160/2052 (0%)]	 KLD Loss: 7.614186 	 NLL Loss: 7705.498535
Train Epoch: 18 [320/2052 (1%)]	 KLD Loss: 32.444508 	 NLL Loss: 14614.883789
Train Epoch: 18 [480/2052 (1%)]	 KLD Loss: 25.191420 	 NLL Loss: 7351.859375
Train Epoch: 18 [640/2052 (2%)]	 KLD Loss: 5.668165 	 NLL Loss: 2528.207520
Train Epoch: 18 [800/2052 (2%)]	 KLD Loss: 6.580755 	 NLL Loss: 3792.726074
Train Epoch: 18 [960/2052 (3%)]	 KLD Loss: 25.991173 	 NLL Loss: 5087.053711
Train Epoch: 18 [1120/2052 (3%)]	 KLD Loss: 18.937611 	 NLL Loss: 67813.671875
Train Epoch: 18 [1280/2052 (4%)]	 KLD Loss: 25.289135 	 NLL Loss: 4065.602295
Train Epoch: 18 [1440/2052 (4%)]	 KLD Loss: 12.512001 	 NLL Loss: 18677.523438
Train Epoch: 18 [1600/2052 (5%)]	 KLD Loss: 83.939514 	 NLL Loss: 370006.968750
Train Epoch: 18 [1760/2052 (5%)]	 KLD Loss: 3.138979 	 NLL Loss: 838.

Train Epoch: 26 [1440/2052 (4%)]	 KLD Loss: 16.385029 	 NLL Loss: 14844.986328
Train Epoch: 26 [1600/2052 (5%)]	 KLD Loss: 83.828964 	 NLL Loss: 362749.187500
Train Epoch: 26 [1760/2052 (5%)]	 KLD Loss: 3.999378 	 NLL Loss: 628.357727
Train Epoch: 26 [1920/2052 (6%)]	 KLD Loss: 33.872543 	 NLL Loss: 12467.224609
Train Epoch: 27 [160/2052 (0%)]	 KLD Loss: 4.846766 	 NLL Loss: 8477.153320
Train Epoch: 27 [320/2052 (1%)]	 KLD Loss: 22.429977 	 NLL Loss: 15778.155273
Train Epoch: 27 [480/2052 (1%)]	 KLD Loss: 28.001608 	 NLL Loss: 6936.653320
Train Epoch: 27 [640/2052 (2%)]	 KLD Loss: 6.612759 	 NLL Loss: 2251.529541
Train Epoch: 27 [800/2052 (2%)]	 KLD Loss: 7.365622 	 NLL Loss: 3456.840332
Train Epoch: 27 [960/2052 (3%)]	 KLD Loss: 17.512079 	 NLL Loss: 3924.201904
Train Epoch: 27 [1120/2052 (3%)]	 KLD Loss: 29.210350 	 NLL Loss: 64773.496094
Train Epoch: 27 [1280/2052 (4%)]	 KLD Loss: 27.671614 	 NLL Loss: 4337.944824
Train Epoch: 27 [1440/2052 (4%)]	 KLD Loss: 16.936377 	 NLL Loss: 145

...

Train Epoch: 62 [160/2052 (0%)]	 KLD Loss: 11.319088 	 NLL Loss: 2610.590332
Train Epoch: 62 [320/2052 (1%)]	 KLD Loss: 31.597019 	 NLL Loss: 11578.381836
Train Epoch: 62 [480/2052 (1%)]	 KLD Loss: 37.351822 	 NLL Loss: 6519.444336
Train Epoch: 62 [640/2052 (2%)]	 KLD Loss: 8.482655 	 NLL Loss: 1794.461548
Train Epoch: 62 [800/2052 (2%)]	 KLD Loss: 9.893488 	 NLL Loss: 2959.339600
Train Epoch: 62 [960/2052 (3%)]	 KLD Loss: 12.952244 	 NLL Loss: 1694.398071
Train Epoch: 62 [1120/2052 (3%)]	 KLD Loss: 34.114975 	 NLL Loss: 62768.683594
Train Epoch: 62 [1280/2052 (4%)]	 KLD Loss: 28.388945 	 NLL Loss: 3992.065186
Train Epoch: 62 [1440/2052 (4%)]	 KLD Loss: 23.441710 	 NLL Loss: 9930.858398
Train Epoch: 62 [1600/2052 (5%)]	 KLD Loss: 339.401917 	 NLL Loss: 81957.398438
Train Epoch: 62 [1760/2052 (5%)]	 KLD Loss: 4.473776 	 NLL Loss: 376.151947
Train Epoch: 62 [1920/2052 (6%)]	 KLD Loss: 44.216049 	 NLL Loss: 7072.578125
Train Epoch: 63 [160/2052 (0%)]	 KLD Loss: 11.565319 	 NLL Loss: 2511.

Train Epoch: 70 [1760/2052 (5%)]	 KLD Loss: 5.112403 	 NLL Loss: 380.130127
Train Epoch: 70 [1920/2052 (6%)]	 KLD Loss: 46.226707 	 NLL Loss: 6647.913086
Train Epoch: 71 [160/2052 (0%)]	 KLD Loss: 13.438473 	 NLL Loss: 2636.244141
Train Epoch: 71 [320/2052 (1%)]	 KLD Loss: 33.445293 	 NLL Loss: 10292.262695
Train Epoch: 71 [480/2052 (1%)]	 KLD Loss: 37.571846 	 NLL Loss: 6306.333008
Train Epoch: 71 [640/2052 (2%)]	 KLD Loss: 8.242517 	 NLL Loss: 1787.404419
Train Epoch: 71 [800/2052 (2%)]	 KLD Loss: 7.920286 	 NLL Loss: 2908.583496
Train Epoch: 71 [960/2052 (3%)]	 KLD Loss: 14.710974 	 NLL Loss: 2326.281250
Train Epoch: 71 [1120/2052 (3%)]	 KLD Loss: 26.016697 	 NLL Loss: 62827.031250
Train Epoch: 71 [1280/2052 (4%)]	 KLD Loss: 21.925159 	 NLL Loss: 3700.112305
Train Epoch: 71 [1440/2052 (4%)]	 KLD Loss: 18.094181 	 NLL Loss: 7352.445312
Train Epoch: 71 [1600/2052 (5%)]	 KLD Loss: 382.663971 	 NLL Loss: 56301.980469
Train Epoch: 71 [1760/2052 (5%)]	 KLD Loss: 5.305840 	 NLL Loss: 365.0

Train Epoch: 79 [1440/2052 (4%)]	 KLD Loss: 21.674313 	 NLL Loss: 7170.306641
Train Epoch: 79 [1600/2052 (5%)]	 KLD Loss: 421.450562 	 NLL Loss: 41054.781250
Train Epoch: 79 [1760/2052 (5%)]	 KLD Loss: 4.412478 	 NLL Loss: 346.918884
Train Epoch: 79 [1920/2052 (6%)]	 KLD Loss: 39.064072 	 NLL Loss: 6468.431641
Train Epoch: 80 [160/2052 (0%)]	 KLD Loss: 13.611105 	 NLL Loss: 2369.324219
Train Epoch: 80 [320/2052 (1%)]	 KLD Loss: 34.023605 	 NLL Loss: 9721.599609
Train Epoch: 80 [480/2052 (1%)]	 KLD Loss: 41.514793 	 NLL Loss: 5751.691406
Train Epoch: 80 [640/2052 (2%)]	 KLD Loss: 9.762292 	 NLL Loss: 1739.406128
Train Epoch: 80 [800/2052 (2%)]	 KLD Loss: 10.481175 	 NLL Loss: 2925.125977
Train Epoch: 80 [960/2052 (3%)]	 KLD Loss: 13.370513 	 NLL Loss: 2667.244141
Train Epoch: 80 [1120/2052 (3%)]	 KLD Loss: 33.872917 	 NLL Loss: 61277.128906
Train Epoch: 80 [1280/2052 (4%)]	 KLD Loss: 28.503614 	 NLL Loss: 3848.431152
Train Epoch: 80 [1440/2052 (4%)]	 KLD Loss: 22.937647 	 NLL Loss: 7431

Train Epoch: 88 [1120/2052 (3%)]	 KLD Loss: 30.857353 	 NLL Loss: 60772.890625
Train Epoch: 88 [1280/2052 (4%)]	 KLD Loss: 23.027000 	 NLL Loss: 3464.516602
Train Epoch: 88 [1440/2052 (4%)]	 KLD Loss: 23.560493 	 NLL Loss: 5717.440430
Train Epoch: 88 [1600/2052 (5%)]	 KLD Loss: 471.769379 	 NLL Loss: 35398.082031
Train Epoch: 88 [1760/2052 (5%)]	 KLD Loss: 4.418157 	 NLL Loss: 305.786713
Train Epoch: 88 [1920/2052 (6%)]	 KLD Loss: 40.826347 	 NLL Loss: 5742.542480
Train Epoch: 89 [160/2052 (0%)]	 KLD Loss: 13.251925 	 NLL Loss: 2011.820312
Train Epoch: 89 [320/2052 (1%)]	 KLD Loss: 34.732834 	 NLL Loss: 8472.835938
Train Epoch: 89 [480/2052 (1%)]	 KLD Loss: 40.288277 	 NLL Loss: 5449.512695
Train Epoch: 89 [640/2052 (2%)]	 KLD Loss: 8.795683 	 NLL Loss: 1688.958252
Train Epoch: 89 [800/2052 (2%)]	 KLD Loss: 8.861351 	 NLL Loss: 2815.557617
Train Epoch: 89 [960/2052 (3%)]	 KLD Loss: 10.195263 	 NLL Loss: 1111.078857
Train Epoch: 89 [1120/2052 (3%)]	 KLD Loss: 33.322277 	 NLL Loss: 60460

Train Epoch: 97 [800/2052 (2%)]	 KLD Loss: 9.148701 	 NLL Loss: 2727.740723
Train Epoch: 97 [960/2052 (3%)]	 KLD Loss: 11.545479 	 NLL Loss: 1169.976074
Train Epoch: 97 [1120/2052 (3%)]	 KLD Loss: 31.828962 	 NLL Loss: 59712.417969
Train Epoch: 97 [1280/2052 (4%)]	 KLD Loss: 26.843922 	 NLL Loss: 3237.654053
Train Epoch: 97 [1440/2052 (4%)]	 KLD Loss: 25.714470 	 NLL Loss: 4895.186523
Train Epoch: 97 [1600/2052 (5%)]	 KLD Loss: 283.932129 	 NLL Loss: 83139.554688
Train Epoch: 97 [1760/2052 (5%)]	 KLD Loss: 4.429502 	 NLL Loss: 243.756561
Train Epoch: 97 [1920/2052 (6%)]	 KLD Loss: 45.254189 	 NLL Loss: 5470.403320
Train Epoch: 98 [160/2052 (0%)]	 KLD Loss: 13.285866 	 NLL Loss: 2011.718506
Train Epoch: 98 [320/2052 (1%)]	 KLD Loss: 39.722221 	 NLL Loss: 8625.367188
Train Epoch: 98 [480/2052 (1%)]	 KLD Loss: 40.160770 	 NLL Loss: 6303.870117
Train Epoch: 98 [640/2052 (2%)]	 KLD Loss: 9.098618 	 NLL Loss: 1664.441406
Train Epoch: 98 [800/2052 (2%)]	 KLD Loss: 10.345669 	 NLL Loss: 2765.9

In [20]:
model.sample(10)

tensor([[ 1.3399e+00,  1.5883e+00,  8.7096e-01,  2.9367e+00, -7.4408e+00,
         -7.1307e-01,  9.7113e+00,  4.8143e-01,  1.6757e+00,  3.9058e+00,
         -4.7724e-01,  5.3615e-01,  4.5017e+00, -8.6025e-01,  1.0346e+00,
         -3.2432e+00, -1.5993e-01, -2.3737e-01,  7.3380e+00, -1.1106e+00,
         -1.9674e-02,  5.4773e+01,  7.3220e+00,  3.1609e+01, -2.3862e-01,
         -7.2630e+00,  1.3165e+02, -2.1035e+01,  1.6367e+01, -2.0119e+01,
          9.2885e+00,  1.1259e+00, -1.2379e+00,  6.0602e+01, -4.2675e+00,
         -2.5331e+01,  1.7882e+00,  1.4614e+00, -1.2955e+02, -2.5077e+01,
         -1.0907e+01,  2.2031e+01,  3.1757e+01, -1.5698e+01,  4.8255e+00,
         -3.3312e+01, -3.9984e+00,  4.2571e+00,  6.8291e-01, -9.2192e+00,
          2.5227e+00,  4.4588e+00, -4.9139e-01,  2.1635e-01, -3.2379e+01,
          3.9923e+00, -6.0985e+00, -1.0502e+00,  8.0443e+00,  5.6476e-01,
          3.1058e+00, -1.5721e-01,  1.5601e-02, -6.8247e-02, -3.1076e-01,
         -1.2861e-01],
        [ 1.185